In [0]:
!pip install --upgrade pip
!pip install tensorflow-gpu

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#Inception
X = np.load("/content/drive/My Drive/VIDEO_RGB/DataFeatures.npy",allow_pickle=True)
Y = np.load("/content/drive/My Drive/VIDEO_RGB/Labels.npy",allow_pickle=True)

#Resnet
# X = np.load("/content/drive/My Drive/VIDEO_RGB/DataFeatures_ResNetV2.npy",allow_pickle=True)
# Y = np.load("/content/drive/My Drive/VIDEO_RGB/Labels_ResNetV2.npy",allow_pickle=True)

In [0]:
import os
directory = "drive/My Drive/VIDEO_RGB"
entries = os.listdir(directory)

num_labels = 12
LabelToVec = {}
VecToLabel = {}
I = np.identity(num_labels);

for i in range(0,num_labels):
  name = entries[i]
  LabelToVec[name] = I[i,:]
  VecToLabel[i] = name

mask = np.arange(num_labels)

Y_vector = []
for i in Y:
  string = str(i,'utf-8')
  vec = LabelToVec[string]
  Y_vector.append(vec)
Y_vector = np.asarray(Y_vector)


In [0]:
X_train = []
Y_train = []

X_val = []
Y_val = []

X_test = []
Y_test = []

for i in range(0,12):
    start = i * 165
    for j in range(0,40):
        X_train.append(X[start + 3*j,:,:])
        X_train.append(X[start + 3*j+1,:,:])
        X_train.append(X[start + 3*j+2,:,:])
        
        Y_train.append(Y_vector[start + 3*j])
        Y_train.append(Y_vector[start + 3*j+1])
        Y_train.append(Y_vector[start + 3*j+2])
        
    for j in range(40,50):
        X_val.append(X[start + 3*j,:,:])
        X_val.append(X[start + 3*j+1,:,:])
        X_val.append(X[start + 3*j+2,:,:])
        
        Y_val.append(Y_vector[start + 3*j])
        Y_val.append(Y_vector[start + 3*j+1])
        Y_val.append(Y_vector[start + 3*j+2])
        
    for j in range(50,55):
        X_test.append(X[start + 3*j,:,:])
        X_test.append(X[start + 3*j+1,:,:])
        X_test.append(X[start + 3*j+2,:,:])
        
        Y_test.append(Y_vector[start + 3*j])
        Y_test.append(Y_vector[start + 3*j+1])
        Y_test.append(Y_vector[start + 3*j+2])
        
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

In [0]:
X_train.shape

(1440, 100, 2048)

In [0]:
import tensorflow as tf
tf.executing_eagerly()
hu1 = 90
hu2 = 90
hu3 = 90
hu4 = 90

timesteps = 100
dataDim = 2048

from keras import regularizers

Using TensorFlow backend.


In [0]:
# 1 layer
# model = tf.keras.Sequential()
# model.add( tf.keras.layers.LSTM(hu1 , input_shape = (timesteps,dataDim),return_sequences=False , kernel_regularizer = regularizers.l2(0.003))  )
# model.add( tf.keras.layers.Dense(12,input_shape=(hu1,),kernel_regularizer=regularizers.l2(0.003)) )
# model.add( tf.keras.layers.Activation('softmax'))

# 2 layer
model = tf.keras.Sequential()
model.add( tf.keras.layers.LSTM(hu1 ,input_shape = (timesteps,dataDim),return_sequences=True, kernel_regularizer = regularizers.l2(0.003)))
model.add( tf.keras.layers.LSTM(hu2 , input_shape = (timesteps,hu1),return_sequences=False , kernel_regularizer = regularizers.l2(0.003))  )
model.add( tf.keras.layers.Dense(12,input_shape=(hu2,),kernel_regularizer=regularizers.l2(0.003)) )
model.add( tf.keras.layers.Activation('softmax'))

# 3 layer
# model = tf.keras.Sequential()
# model.add( tf.keras.layers.LSTM(hu1 , input_shape = (timesteps,dataDim),return_sequences=True , kernel_regularizer = regularizers.l2(0.003))  )
# model.add( tf.keras.layers.LSTM(hu2 ,input_shape = (timesteps,hu1),return_sequences=True, kernel_regularizer = regularizers.l2(0.003)))
# model.add( tf.keras.layers.LSTM(hu3 , input_shape = (timesteps,hu2),return_sequences=False , kernel_regularizer = regularizers.l2(0.003))  )
# model.add( tf.keras.layers.Dense(12,input_shape=(hu3,),kernel_regularizer=regularizers.l2(0.003)) )
# model.add( tf.keras.layers.Activation('softmax'))

In [0]:
lr = 0.001 / 0.95

In [0]:
for epocs in range(0,400):
# for epocs in range(0,270):

  if (epocs % 50==0) :
    lr = lr * 0.95
    adam = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=True) 
    model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=[tf.keras.metrics.CategoricalAccuracy()])
     
  print("Iteration Number", epocs)
  model.fit(X_train,Y_train, epochs=1 ,batch_size = 32, shuffle = True, steps_per_epoch = 45)
  out = model(X_val)
  out = out.numpy()
  l = len(Y_val)
  correct = 0
  total = 0
  for i in range(0,l):
      a = np.argmax(out[i])
      b = np.argmax(Y_val[i])
      if (a==b) : correct = correct + 1
      total = total + 1

  print("\n Validation Accuracy",correct/total)

Iteration Number 0
Train on 1440 samples
1440/1440 [==============================] - 5s 4ms/sample - loss: 3.5778 - categorical_accuracy: 0.0924

 Validation Accuracy 0.14722222222222223
Iteration Number 1
Train on 1440 samples
1440/1440 [==============================] - 1s 840us/sample - loss: 2.7025 - categorical_accuracy: 0.1243

 Validation Accuracy 0.10277777777777777
Iteration Number 2
Train on 1440 samples
1440/1440 [==============================] - 1s 806us/sample - loss: 2.5780 - categorical_accuracy: 0.1111

 Validation Accuracy 0.13333333333333333
Iteration Number 3
Train on 1440 samples
1440/1440 [==============================] - 1s 796us/sample - loss: 2.5468 - categorical_accuracy: 0.1118

 Validation Accuracy 0.13333333333333333
Iteration Number 4
Train on 1440 samples
1440/1440 [==============================] - 1s 813us/sample - loss: 2.5135 - categorical_accuracy: 0.1194

 Validation Accuracy 0.10833333333333334
Iteration Number 5
Train on 1440 samples
1440/1440 [

In [0]:
confusion_matrix = np.zeros((12,12))

out = model(X_test)
out = out.numpy()
l = len(Y_test)
alist = []
blist = []
correct = 0
total = 0
for i in range(0,l):
      a = np.argmax(out[i])
      b = np.argmax(Y_test[i])
      alist.append(a)
      blist.append(b)
      confusion_matrix[b][a] += 1
      
confusion_matrix

array([[ 4.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  0.,  0.,  2.,  3.],
       [ 0.,  5.,  1.,  3.,  1.,  0.,  2.,  0.,  0.,  3.,  0.,  0.],
       [ 0.,  0.,  8.,  0.,  5.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  0., 10.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  9.,  2.,  0.,  0.,  0.,  0.,  2.,  0.],
       [ 0.,  0.,  2.,  0.,  3., 10.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  5.,  0.,  1.,  0.,  7.,  0.,  1.,  0.,  0.,  1.],
       [ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  2.,  6.],
       [ 0.,  4.,  2.,  0.,  1.,  0.,  4.,  0.,  3.,  1.,  0.,  0.],
       [ 0.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  3.,  8.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  1.,  0.,  9.,  2.],
       [ 2.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,  2.,  0.,  1.,  4.]])

In [0]:
s = np.sum(confusion_matrix, axis=1)
print(s)
for i in range(12):
  confusion_matrix[i] /= s[i]

sum = 0.

for i in range(12) :
  print(VecToLabel[i])
  print(confusion_matrix[i][i])
  sum += confusion_matrix[i][i]

print(sum/12)

[15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.]
kick_service
0.26666666666666666
forehand_slice
0.3333333333333333
backhand2hands
0.5333333333333333
forehand_volley
0.6666666666666666
forehand_flat
0.6
forehand_openstands
0.6666666666666666
backhand
0.4666666666666667
flat_service
0.26666666666666666
backhand_slice
0.2
backhand_volley
0.5333333333333333
smash
0.6
slice_service
0.26666666666666666
0.44999999999999996


In [0]:

#Inception

# np.savetxt("drive/My Drive/RESULTS/LSTM/Inception/Confusion_Matrix_Inception_LSTM_1layer.csv",confusion_matrix,delimiter = ",")
np.savetxt("drive/My Drive/RESULTS/LSTM/Inception/Confusion_Matrix_Inception_LSTM_2layer.csv",confusion_matrix,delimiter = ",")
# np.savetxt("drive/My Drive/RESULTS/LSTM/Inception/Confusion_Matrix_Inception_LSTM_3layer.csv",confusion_matrix,delimiter = ",")

#Resnet

# np.savetxt("drive/My Drive/RESULTS/LSTM/Resnet/Confusion_Matrix_ResNet_LSTM_1layer.csv",confusion_matrix,delimiter = ",")
# np.savetxt("drive/My Drive/RESULTS/LSTM/Resnet/Confusion_Matrix_ResNet_LSTM_2layer.csv",confusion_matrix,delimiter = ",")
# np.savetxt("drive/My Drive/RESULTS/LSTM/Resnet/Confusion_Matrix_ResNet_LSTM_3layer.csv",confusion_matrix,delimiter = ",")